In [7]:
from collections import defaultdict
from heapq import heappop, heappush
import re

sample = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II""".splitlines()

def parse(lines)->dict[str,tuple[int,list[str]]]:
    nodes = {}
    for line in lines:
        x = re.findall("Valve (.+) has flow rate=(.+); .*valves? (.*)", line)
        [(node, flow, dests)] = x
        dests_split = dests.split(', ')
        nodes[node] = (int(flow), dests_split)
    return nodes

def find_costs(nodes:dict[str,tuple[int,list[str]]]):
    costs:dict[str,list[dict]] = {}
    for start in nodes.keys():
        visited = {}
        heap = []
        heappush(heap, (0,0,start,[]))
        while len(heap)>0:
            length, current_cost, node, path = heappop(heap)
            if node in visited.keys(): continue
            visited[node] = (length, current_cost, path)
            _, neighbors = nodes[node]            
            for n in neighbors:
                n_flow, _ = nodes[n]
                new_length = length + 1
                if n_flow > 0: new_length += 1
                #print(f"{start}->{n} {current_cost=} new_cost={current_cost+flow + n_flow}")
                heappush(heap, (length+2, current_cost + n_flow, n, path + [n]))
        del visited[start]
        costs[start] = visited
    
    for node, cost in costs.items():
        print(f"{node} -> {cost}")
    return costs

def travel(time_limit, total_cost, nodes:dict[str,tuple[int,list[str]]], costs:dict[str,list[dict]]):
    time, flow, path = total_cost

    # if full, return total cost
    if len(path) == len(nodes): return total_cost

    # start at path[-1]
    curr = path[-1]

    visited = set(path)
    other_nodes = set(nodes.keys()) - visited

    highest_cost = (0,0)
    # for each other node
    for other_node in other_nodes:
        # skip it if we've been there
        if other_node in visited: continue

        print(f"try {path} -> {other_node}")

        other_time, other_flow, other_path = costs[curr][other_node]

        #print(time,other_time, flow,other_flow, path,other_path)
        new_cost = (time+other_time, flow+other_flow, path+other_path)

        if time > time_limit: continue  # if we're over the limit, bail

        new_path = new_cost[2]
        if len(new_path) > len(set(new_path)): continue  # dupe? bail

        print(f"{new_cost=}")

        with_node = travel(time_limit, new_cost, nodes, costs)

        # # try without it
        # without_node = travel(time_limit, total_cost, path, nodes, costs)

        # update the cost
        old_highest_cost = highest_cost
        if highest_cost[1] < with_node[1]: highest_cost = with_node
        print(highest_cost, with_node, old_highest_cost)
    
    return highest_cost

def part1(nodes:dict[str,tuple[int,list[str]]], costs:dict[str,list[dict]]):
    return travel(30, (0,0,['AA']), nodes, costs)
    

nodes = parse(sample)
costs = find_costs(nodes)
print(part1(nodes, costs))


AA -> {'II': (2, 0, ['II']), 'BB': (2, 13, ['BB']), 'DD': (2, 20, ['DD']), 'CC': (4, 15, ['BB', 'CC']), 'JJ': (4, 21, ['II', 'JJ']), 'EE': (4, 23, ['DD', 'EE']), 'FF': (6, 23, ['DD', 'EE', 'FF']), 'GG': (8, 23, ['DD', 'EE', 'FF', 'GG']), 'HH': (10, 45, ['DD', 'EE', 'FF', 'GG', 'HH'])}
BB -> {'AA': (2, 0, ['AA']), 'CC': (2, 2, ['CC']), 'II': (4, 0, ['AA', 'II']), 'DD': (4, 20, ['AA', 'DD']), 'JJ': (6, 21, ['AA', 'II', 'JJ']), 'EE': (6, 23, ['AA', 'DD', 'EE']), 'FF': (8, 23, ['AA', 'DD', 'EE', 'FF']), 'GG': (10, 23, ['AA', 'DD', 'EE', 'FF', 'GG']), 'HH': (12, 45, ['AA', 'DD', 'EE', 'FF', 'GG', 'HH'])}
CC -> {'BB': (2, 13, ['BB']), 'DD': (2, 20, ['DD']), 'AA': (4, 13, ['BB', 'AA']), 'EE': (4, 23, ['DD', 'EE']), 'II': (6, 13, ['BB', 'AA', 'II']), 'FF': (6, 23, ['DD', 'EE', 'FF']), 'GG': (8, 23, ['DD', 'EE', 'FF', 'GG']), 'JJ': (8, 34, ['BB', 'AA', 'II', 'JJ']), 'HH': (10, 45, ['DD', 'EE', 'FF', 'GG', 'HH'])}
DD -> {'AA': (2, 0, ['AA']), 'CC': (2, 2, ['CC']), 'EE': (2, 3, ['EE']), 'II': (4,